# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv("output_data/cities.csv")
cities_df = pd.DataFrame(cities)
del cities_df ['Unnamed: 0']
cities_df

,City Name,Temperature,Humidity,Latitude,Longitude,Cloudiness,Wind Speed,Country
0,Chekhov,-1.00,92,55.14,37.47,75,4.00,RU
1,Rikitea,26.37,79,-23.12,-134.97,75,6.53,PF
2,Riyadh,9.00,39,24.69,46.72,0,3.10,SA
3,Bariloche,12.00,58,-41.15,-71.31,20,8.70,AR
4,Tasiilaq,-5.00,79,65.61,-37.64,100,9.80,GL
...,...,...,...,...,...,...,...,...
551,Sebring,16.11,87,27.50,-81.44,1,4.10,US
552,Klamath Falls,11.00,34,42.22,-121.78,1,6.70,US
553,Challakere,23.26,55,14.32,76.65,20,6.12,IN
554,Norilsk,-35.00,73,69.35,88.20,0,2.00,RU


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Latitude", "Longitude"]].astype(float)
# Humidity weight 
humidity = cities_df["Humidity"].astype(int)


In [5]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the DataFrame to find your ideal weather condition. 
# I like Humidity levels between 45 and 50%.

# Create a new DataFrame
ideal_cities =  cities_df[ (cities_df["Humidity"]> 45) & (cities_df["Humidity"] <50)]

ideal_cities 




,City Name,Temperature,Humidity,Latitude,Longitude,Cloudiness,Wind Speed,Country
123,Juegang,15.56,46,32.32,121.19,0,2.37,CN
207,Los Llanos de Aridane,20.00,48,28.66,-17.92,0,2.60,ES
234,Sao Filipe,23.92,48,14.90,-24.50,0,4.23,CV
283,Yumen,-6.08,46,40.28,97.20,0,4.54,CN
334,Doaba,3.88,48,33.42,70.74,0,0.88,PK
356,Murwillumbah,34.44,46,-28.32,153.40,0,6.20,AU
412,Punta Alta,27.32,46,-38.88,-62.07,86,8.12,AR
475,Vung Tau,28.49,46,10.35,107.08,0,1.62,VN
488,Pishin,2.50,49,30.58,67.00,0,1.15,PK
524,Hecun,6.32,47,36.53,114.11,0,2.21,CN


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = ideal_cities
hotel_name =[]
hotel_df['Hotel Name']=''

#hotel_name.append(response['name'])



C:\Users\Olga\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    long = row["Longitude"]
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{long}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # make request and print url
    target_url_response = requests.get(base_url, params=params)
    # convert to json
    target_url_response = target_url_response.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    try:
        hotel_name.append(target_url_response['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

hotel_df['Hotel Name']= hotel_name


C:\Users\Olga\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
hotel_df = hotel_df.dropna()
hotel_df.to_csv('hotel_export.csv')
hotel_df

,City Name,Temperature,Humidity,Latitude,Longitude,Cloudiness,Wind Speed,Country,Hotel Name
123,Juegang,15.56,46,32.32,121.19,0,2.37,CN,Landison Hotel Nantong
207,Los Llanos de Aridane,20.00,48,28.66,-17.92,0,2.60,ES,Hotel Hacienda de Abajo
234,Sao Filipe,23.92,48,14.90,-24.50,0,4.23,CV,Casa Colonial Koenig
334,Doaba,3.88,48,33.42,70.74,0,0.88,PK,Mian hayat khan
356,Murwillumbah,34.44,46,-28.32,153.40,0,6.20,AU,Tweed River Motel
412,Punta Alta,27.32,46,-38.88,-62.07,86,8.12,AR,Hotel Barcelona Center - Restaurant & Confitería
475,Vung Tau,28.49,46,10.35,107.08,0,1.62,VN,The IMPERIAL Hotel
488,Pishin,2.50,49,30.58,67.00,0,1.15,PK,New haji Shah Muhammad Hotel
535,Nishihara,18.00,48,26.18,127.76,20,1.50,JP,百名伽藍


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [12]:
# Add marker layer ontop of heat map
layer = gmaps.symbol_layer(locations,info_box_content=hotel_info)
fig.add_layer(layer)
fig


# Display Map

Figure(layout=FigureLayout(height='420px'))